In [ ]:
import torch
from transformers import AutoformerForPrediction
from huggingface_hub import hf_hub_download

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
version = "huggingface/autoformer-tourism-monthly"
repo_id = "hf-internal-testing/tourism-monthly-batch"

# file

In [ ]:
file = hf_hub_download(
    repo_id="hf-internal-testing/tourism-monthly-batch",
    filename="train-batch.pt",
    repo_type="dataset",
)
file

In [ ]:
batch = torch.load(file)
batch

# AutoformerForPrediction

In [ ]:
model = AutoformerForPrediction.from_pretrained(version)
model

## train

In [ ]:
# during training, one provides both past and future values
# as well as possible additional features
outputs = model(
    past_values=batch["past_values"],
    past_time_features=batch["past_time_features"],
    past_observed_mask=batch["past_observed_mask"],
    static_categorical_features=batch["static_categorical_features"],
    static_real_features=batch["static_real_features"],
    future_values=batch["future_values"],
    future_time_features=batch["future_time_features"],
)
outputs.loss

In [ ]:
outputs.last_hidden_state

## predict

In [ ]:
model.eval()
with torch.inference_mode():
    # during inference, one only provides past values
    # as well as possible additional features
    # the model autoregressively generates future values
    outputs = model.generate(
        past_values=batch["past_values"],
        past_time_features=batch["past_time_features"],
        past_observed_mask=batch["past_observed_mask"],
        static_categorical_features=batch["static_categorical_features"],
        static_real_features=batch["static_real_features"],
        future_time_features=batch["future_time_features"],
    )
outputs

In [ ]:
mean_prediction = outputs.sequences.mean(dim=1)
mean_prediction